## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self._precio = 0  
        self._cantidad = 0
        self.precio = precio  
        self.cantidad = cantidad  

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, value):
        if value >= 0:
            self._precio = value

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, value):
        if value >= 0:
            self._cantidad = value

    def valorTotal(self):
        return self.cantidad * self.precio

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigos=None, tipos=None):
        self.descripcion = descripcion
        self.codigos = codigos or []  
        self.tipos = tipos or [] 

    def es_aplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
           super().__init__(f"Descuento del {descuento}%", *args, **kwargs)  
           self.descuento = descuento 

    def calcularDescuento(self, producto, cantidad):
        if self.es_aplicable(producto):
            return producto.precio * cantidad * (self.descuento / 100)
        return 0
    
class Oferta2x1(Oferta):
    def __init__(self, *args, **kwargs):
        super().__init__("Oferta 2x1", *args, **kwargs)  

    def calcularDescuento(self, producto, cantidad):
        """Calcula el descuento de 2x1 si el producto es aplicable y la cantidad es 2 o más."""
        if self.es_aplicable(producto) and cantidad >= 2:
            return producto.precio * (cantidad // 2)  
        return 0

In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        self.productos.extend(productos)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.precio * p.cantidad for p in self.productos)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscar(self, codigo):
        return next((producto for producto in self.productos if producto.codigo == codigo), None)

    def calcularDescuento(self, producto, cantidad):
        return max((oferta.calcularDescuento(producto, cantidad) for oferta in self.ofertas), default=0)

    def guardar(self, filename):
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    @classmethod
    def leer(cls, filename):
        catalogo = cls()
        with open(filename, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                producto = Producto(row['codigo'], row['nombre'], float(row['precio']), row['tipo'], int(row['cantidad']))
                catalogo.agregar(producto)
        return catalogo

    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad
        else:
            raise ValueError("No hay suficiente stock disponible")

    def informe(self):
        informe = [
            f"Cantidad de productos: {self.cantidadProductos}",
            f"Cantidad de unidades: {self.cantidadUnidades}",
        ]

        if self.cantidadUnidades > 0:
            informe.append(f"Precio Promedio: {self.valorTotal / self.cantidadUnidades:.2f}")

        informe.append(f"Valor total: {self.valorTotal:.2f}")
        informe.append(f"Tipos de productos: {', '.join(set(p.tipo for p in self.productos))}")
        informe.append("Ofertas:")

        for oferta in self.ofertas:
            informe.append(f"  {oferta.descripcion}")

        return "\n".join(informe)


In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if self._validar_nombre(value):
            self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        if self.validar_cuit(value):
            self._cuit = value

    @staticmethod
    def validar_cuit(cuit):
        return re.match(r'^\d{2}-\d{8}-\d$', cuit) is not None

    @staticmethod
    def _validar_nombre(nombre):
        return 1 <= len(nombre) <= 100

In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [9]:
from datetime import datetime

class Factura:
    contador = 100  

    @classmethod
    def ultimaFactura(cls, num):
        cls.contador = num

    @classmethod
    def nuevoNumero(cls):
        cls.contador += 1
        return cls.contador

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.productos = []

    def agregar(self, producto, cantidad):
        self.catalogo.vender(producto, cantidad)
        self._agregarProducto(producto, cantidad)

    def _agregarProducto(self, producto, cantidad):
        for p in self.productos:
            if p['producto'].codigo == producto.codigo:
                p['cantidad'] += cantidad
                return
        self.productos.append({'producto': producto, 'cantidad': cantidad})

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p['cantidad'] for p in self.productos)

    @property
    def subtotal(self):
        return sum(p['producto'].precio * p['cantidad'] for p in self.productos)

    @property
    def descuentos(self):
        return sum(self.catalogo.calcularDescuento(p['producto'], p['cantidad']) for p in self.productos)

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        return self._formatearFactura()

    def _formatearFactura(self):
        impresion = f"Factura: {self.numero}\n"
        impresion += f"Fecha  : {self.fecha}\n"
        impresion += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"

        for p in self.productos:
            producto = p['producto']
            cantidad = p['cantidad']
            subtotal_producto = producto.precio * cantidad
            impresion += f"- {cantidad} {producto.nombre:<30} x ${producto.precio:.2f} = ${subtotal_producto:.2f}\n"
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            
            if descuento > 0:
                impresion += f"      Descuento 10%                             - ${descuento:.2f}\n"
                if any(isinstance(oferta, Oferta2x1) for oferta in self.catalogo.ofertas):
                    impresion += f"      Oferta 2x1                                - ${descuento:.2f}\n"

        impresion += f"{'-' * 45}\n"
        impresion += f"{'Subtotal:':>39} ${self.subtotal:.2f}\n"
        impresion += f"{'Descuentos:':>39} ${self.descuentos:.2f}\n"
        impresion += f"{'TOTAL:':>39} ${self.total:.2f}\n"

        return impresion


In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
